In [3]:
import pandas as pd
import unicodedata
from difflib import SequenceMatcher

# Função para normalizar texto: remove acentos, põe em minúsculo e tira pontuação
def normalize_text(text):
    if not isinstance(text, str):
        return ''
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return ''.join(e for e in text.lower() if e.isalnum() or e.isspace()).strip()

# Função para calcular similaridade entre duas strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# --- LEITURA DOS ARQUIVOS ---
df_vendas = pd.read_csv("Book(Planilha4).csv",sep=';', encoding='utf-8')   # Deve ter uma coluna chamada "Título"
df_books = pd.read_csv("Book(CATALOGO).csv",sep=';', encoding='utf-8')     # Deve ter colunas "TITULO" e "ISBN"

# --- NORMALIZAÇÃO DOS TEXTOS ---
df_vendas["Título_norm"] = df_vendas["Título"].apply(normalize_text)
df_books["TITULO_norm"] = df_books["TITULO"].apply(normalize_text)

# Inicializa a nova coluna
df_vendas["ISBN"] = None

# --- CRUZAMENTO COM SIMILARIDADE ---
for i, row_venda in df_vendas.iterrows():
    melhor_score = 0
    melhor_isbn = None
    for _, row_book in df_books.iterrows():
        score = similar(row_venda["Título_norm"], row_book["TITULO_norm"])
        if score > melhor_score:
            melhor_score = score
            melhor_isbn = row_book["ISBN"]
    if melhor_score >= 0.8:  # Aceita parecidos com pelo menos 80%
        df_vendas.at[i, "ISBN"] = melhor_isbn

# --- SALVAR RESULTADO ---
df_vendas[["Título", "ISBN"]].to_csv("vendas_com_isbns.csv", index=False)
print("Arquivo 'vendas_com_isbns.csv' gerado com sucesso!")


Arquivo 'vendas_com_isbns.csv' gerado com sucesso!


In [5]:
df = pd.read_csv("Vendas_com_ISBNs.csv")
df.head(20)

,Título,ISBN
0,"ASCENSAO DO RESTO, A",9.788571e+12
1,"CIÊNCIA, EDUCAÇÃO E LIVROS DIDÁTICOS DO SÉCULO...",NaN
2,Promocional,9.780000e+12
3,Uma estratégia para o Ocidente: o conceito de ...,NaN
4,1a ed. TEMAS ATUAIS EM DIDATICA,9.788570e+12
5,Discussões tusculanas,9.788571e+12
6,promocional,9.780000e+12
7,As leis,NaN
8,PODE O SUBALTERNO FALAR?,9.788570e+12
9,SOCIOLOGIA DO DESCONHECIMENTO,9.786557e+12


In [7]:
df.to_excel("Vendas.xlsx")